In [4]:
import pandas as pd 
import datetime
import numpy as np
import matplotlib.pyplot as plt
import re
import os
%matplotlib inline


# Предобработка и соединение файлов с номерами и датами

In [5]:
quart = pd.DataFrame()

for file in os.listdir():
    if '$' not in file and 'xlsx' in file and 'АУ.xlsx' not in file:
        dict_df = pd.read_excel(file, sheet_name=None, dtype=str)
        for key in dict_df.keys():
            name = (file[file.find('.') + 1:file.rfind('.')] + ' ' + key).strip()
            temp_df = dict_df[key][['Дата', 'Клиент: телефон']]
            temp_df['Источник'] = [name] * temp_df.shape[0]
            quart = quart.append(temp_df)

C:\Users\AmanovRA\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [6]:
quart = quart.reset_index(drop=True)


In [7]:
pattern = re.compile(r'([^ ]+)( [^ ]+)')


def string(s):
    pattern = re.compile(r'([^ ]+)( [^ ]+)')
    if pattern.findall(s):
        return pattern.findall(s)[0][0]
    else:
        return s
    
quart['Дата_new'] = quart['Дата'].apply(string)


In [8]:
def to_date(x):
    try:
        x = datetime.datetime.strptime(x, '%Y-%m-%d').date()
    except:
        x = datetime.datetime.strptime(x, '%d.%m.%Y').date()
    return x    

In [9]:
quart['Дата_new'] = quart['Дата_new'].apply(to_date)

In [10]:
quart.drop('Дата', axis=1, inplace=True)

In [11]:
quart = quart.rename(columns={'Дата_new': 'Дата'})

In [12]:
quart

,Клиент: телефон,Источник,Дата
0,79062772319,ТС - Соцсети Звонки,2020-12-26
1,79213000190,ТС - Соцсети Звонки,2020-12-10
2,79112597604,ТС - Соцсети Звонки,2020-12-08
3,79633247371,ТС - Соцсети Звонки,2020-11-24
4,79650395604,ТС - Соцсети Звонки,2020-11-02
...,...,...,...
1457,8-906-267-48-04,ТС - Все Заявки,2020-10-21
1458,8-911-183-61-31,ТС - Все Заявки,2020-10-21
1459,8-911-183-61-31,ТС - Все Заявки,2020-10-21
1460,8-921-798-94-43,ТС - Все Заявки,2020-10-21


# re - телефоны

In [13]:
phones = quart['Клиент: телефон'].tolist()

pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for i, j in enumerate(phones):
    if len([x for x in j if x.isdigit()]) == 10:
        phones[i] = '7' + ''.join([x for x in j if x.isdigit()])    
    if len([x for x in j if x.isdigit()]) == 11:
        temp_arr = ''.join([x for x in j if x.isdigit()])
        if temp_arr[0] == '8':
            phones[i] = '7' + temp_arr[1:]
        elif temp_arr[0] == '7':    
            phones[i] = temp_arr

for i, j in enumerate(phones):
    if re.search(pattern, j):
        if '(61)' in j:
            phones[i] = phones[i].replace('(61)', ', 321-61-61, ')
        elif  '(39)' in j:   
            phones[i] = phones[i].replace('(39)', ', 305-73-39, ')
        elif '(73)' in j:
            phones[i] = phones[i].replace('(73)', '')
            phones[i] += ' , 558-73-09 '
        elif ('21') in j:
            phones[i] = phones[i].replace('(21)', '(921)')
    phones[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
    phones[i] = phones[i].lstrip(' ')
    phones[i] = phones[i].rstrip(' ')
    if re.search(pattern2, phones[i]):
        phones[i] = re.sub(pattern2, '', phones[i])
    phones[i] = ''.join([x for x in phones[i] if x not in '()-']) 
    
phones2 = phones.copy()

for i, j in enumerate(phones2):
    phones2[i] = j.split()
    
from functools import reduce
for i, j in enumerate(phones2):
    try:
        phones2[i].append(reduce(lambda x, y: x + y, [z for z in j if len(z) < 7]))
    except:
        continue
        
for i, j in enumerate(phones2):
    for x, y in enumerate(j):
        if x != len(j) - 1:
            if len(y) == 4 and len(j[x + 1]) == 7 and y[:2] in ('79', '89'):
                    phones2[i].append(y + j[x + 1])
            elif y in ('7', '8') and len(j[x + 1]) == 10:
                phones2[i].append(y + j[x + 1])
                
for i, j in enumerate(phones2):
    phones2[i] = [x for x in phones2[i] if len(x) >= 7]
    
for i, j in enumerate(phones2):
    for x, y in enumerate(j):
        if len(y) == 7:
            phones2[i][x] = '7812' + phones2[i][x]
        elif len(y) == 11 and y[0] == '8':
            phones2[i][x] = '7' + y[1:]
        elif len(y) == 10:
            phones2[i][x] = '7' + y    

In [14]:
quart['длина'] = quart['Клиент: телефон'].apply(len)

In [15]:
quart['Чистые телефоны'] = phones2

In [16]:
quart['Чистые телефоны'] = quart['Чистые телефоны'].apply(lambda x: ''.join(x) if isinstance(x, list) else x)

In [17]:
quart['длина чистые'] = quart['Чистые телефоны'].apply(len)

In [25]:
quart[quart['длина чистые'] != 11]






,Клиент: телефон,Источник,Дата,длина,Чистые телефоны,длина чистые,соответствие


In [19]:
def clean_number(row):
    if row['длина чистые'] == 14:
        row['Чистые телефоны'] = '7' + row['Чистые телефоны'][4:]
    return row
        

In [20]:
quart = quart.apply(clean_number, axis=1)

In [21]:
quart['длина чистые'] = quart['Чистые телефоны'].apply(len)

In [22]:
pattern_7 = r'^7\d{10}$'
quart['соответствие'] = quart['Чистые телефоны'].apply(lambda x: bool(re.match(pattern_7, x)))

In [23]:
quart[~quart['соответствие']]

,Клиент: телефон,Источник,Дата,длина,Чистые телефоны,длина чистые,соответствие
108,41227351142,ТС - Все Звонки,2020-12-27,11,41227351142,11,False


In [24]:
quart.drop([108], inplace=True)


In [26]:
quart.drop(['Клиент: телефон', 'длина', 'длина чистые', 'соответствие'], axis=1, inplace=True)

In [27]:
quart = quart.sort_values(by='Дата', ignore_index=True)
quart = quart.drop_duplicates(subset='Чистые телефоны', ignore_index=True)
quart

,Источник,Дата,Чистые телефоны
0,ТС - Все Звонки,2020-10-01,79216582058
1,ТС - Все Звонки,2020-10-01,79818757402
2,ТС - Все Звонки,2020-10-01,79218966906
3,ТС - Все Звонки,2020-10-01,79658139663
4,ТС - Все Звонки,2020-10-01,79119910046
...,...,...,...
1258,ТС - Все Заявки,2020-12-31,79117715148
1259,ТС - Все Заявки,2020-12-31,79117716860
1260,ТС - Все Звонки,2020-12-31,79819579567
1261,ТС - Все Заявки,2020-12-31,79531628374


# АУ

In [28]:
au_telefons = pd.read_excel('АУ.xlsx')
au_telefons = au_telefons.fillna('')

In [29]:
dom, mobil, work = au_telefons['Телефон домашний'].tolist(), au_telefons['Телефон мобильный'].tolist(), au_telefons['Телефон рабочий'].tolist()

In [30]:
pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if len([x for x in j if x.isdigit()]) == 10:
            key[i] = '7' + ''.join([x for x in j if x.isdigit()])    
        if len([x for x in j if x.isdigit()]) == 11:
            temp_arr = ''.join([x for x in j if x.isdigit()])
            if temp_arr[0] == '8':
                key[i] = '7' + temp_arr[1:]
            elif temp_arr[0] == '7':    
                key[i] = temp_arr




for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            key[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
            key[i] = key[i].lstrip(' ')
            key[i] = key[i].rstrip(' ')
            if re.search(pattern2, key[i]):
                key[i] = re.sub(pattern2, '', key[i])
            key[i] = ''.join([x for x in key[i] if x not in '()-'])  

In [31]:
for key in (dom, mobil, work):
    for i, j in enumerate(key):
        if type(j) == str:
            if len(j) == 7:
                key[i] = '7812' + key[i]
            elif len(j) == 11 and j[0] == '8':
                key[i] = '7' + j[1:]
            elif len(j) == 10:
                key[i] = '7' + j

In [32]:
def concate_lists(*args) -> list:   # соединим все номера в одну строку чтобы проще было искать
    temp_lst = []
    for i in range(len(args[0])):
        temp_lst.append(' '.join([str(x[i]) for x in args]))
    return temp_lst
            
lst_search = concate_lists(dom, mobil, work)        

In [33]:
lst_search

['78123688345 79219810994 79219142652',
 '78123461414 78129956897 79219956897',
 ' 79217539192 ',
 '78123688345 79219810994 79219142652',
 '78123688345 79219810994 79219142652',
 '78123688345 79219810994 79219142652',
 '78123688345 79219810994 79219142652',
 '78123688345 79219810994 79219142652',
 '  ',
 '  ',
 '  ',
 '  ',
 '  ',
 '79990377603 79213075614 ',
 ' 79216337171 ',
 '79990377603 79213075614 ',
 '79119272194 79119676394 ',
 ' 79138568989 ',
 ' 79045537500 ',
 ' 79045537500 ',
 ' 79045537500 ',
 ' 79045537500 ',
 ' 79045537500 ',
 ' 79045537500 ',
 ' 79045537500 ',
 ' 79045537500 ',
 ' 79045537500 ',
 ' 79045537500 ',
 ' 79045537500 ',
 ' 79045537500 ',
 ' 79045537500 ',
 ' 79045537500 ',
 ' 79045537500 ',
 ' 79522025711 ',
 ' 79774896398 ',
 ' 79112585498 79042299738',
 '78122952779 79219025744 78122954001',
 '78122952779 79219025744 78122954001',
 '78122952779 79219025744 78122954001',
 ' 79219071416 ',
 '78122952779 79219025744 78122954001',
 '78122952779 79219025744 78122

# Сравнение

In [34]:
lst_with_coincidenc = []
for index, row in enumerate(lst_search):
    lst_with_coincidenc.append('')
    for i in quart['Чистые телефоны']:
        if str(i) in row:
            lst_with_coincidenc[index] = str(i)
            break
            
            
    

In [35]:
au_telefons['Совпадение'] = lst_with_coincidenc

In [36]:
au_telefons_final = au_telefons[au_telefons['Совпадение'] != '']

In [37]:
au_telefons_final = au_telefons_final.merge(quart, left_on='Совпадение', right_on='Чистые телефоны', how='left')

In [38]:
au_telefons_final = au_telefons_final.fillna('')

In [39]:
au_telefons_final['Старый/Новый'] = au_telefons_final['Дата приема'] == au_telefons_final['Минимальная дата приема Пациента (без удаленных)']
au_telefons_final['Старый/Новый'] = au_telefons_final['Старый/Новый'].replace({True:'Новый', False:'Старый'})
new_clients_lst = au_telefons_final[au_telefons_final['Старый/Новый'] == 'Новый']['Номер карты'].unique()
au_telefons_final.drop('Старый/Новый', axis=1, inplace=True)
au_telefons_final['Старый/Новый'] = au_telefons_final['Номер карты'].isin(new_clients_lst)
au_telefons_final['Старый/Новый'] = au_telefons_final['Старый/Новый'].replace({True:'Новый', False:'Старый'})


In [40]:
au_telefons_final

,Номер карты,Пациент,Дата приема,Минимальная дата приема Пациента (без удаленных),Специализация АУ_ОСН,Телефон мобильный,Телефон домашний,Телефон рабочий,Категория клиента ПЭО,Sum([Стоимость полная]),Sum([Количество]),Филиал,Совпадение,Источник,Дата,Чистые телефоны,Старый/Новый
0,990535952,Поляков Юрий Павлович,2020-11-13,2020-11-13,нет,-,+7(812)424-42-03,-,СК факт,2500.0,1,БДВУ,78124244203,ТС - Все Звонки,2020-12-23,78124244203,Новый
1,990545257,Тиунова Ирина Сергеевна,2020-12-29,2020-12-29,нет,-,-,+7(921)423-60-53,-,12250.0,2,БДВУ,79214236053,ТС - Все Заявки,2020-12-28,79214236053,Новый
2,990551332,Кузнецова Лариса Александровна,2020-12-29,2020-12-29,нет,+7(921)904-27-74,-,-,-,9000.0,2,БДВУ,79219042774,ТС - Все Звонки,2020-12-29,79219042774,Новый
3,990523705,Багрова Елена Николаевна,2020-11-08,2020-09-20,нет,+7(911)184-99-70,,,-,6250.0,2,БДВУ,79111849970,ТС - Все Звонки,2020-10-26,79111849970,Старый
4,990534557,Борисов Дмитрий Анатольевич,2020-11-18,2020-11-06,нет,+7(904)646-00-81,+7(911)985-45-95,-,-,6125.0,2,БДВУ,79119854595,ТС - Все Звонки,2020-11-05,79119854595,Новый
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,120566717,Саадуева Ирина Петровна,2020-10-09,2019-06-10,нет,+7(981)784-35-74,,,-,500.0,1,БДВУ,79817843574,ТС - Все Заявки,2020-10-20,79817843574,Старый
288,990433928,Константинова Клавдия Федоровна,2020-11-13,2019-04-20,нет,,+7(921)902-69-52,,-,460.0,1,БДВУ,79219026952,ТС - Все Звонки,2020-12-27,79219026952,Старый
289,990433928,Константинова Клавдия Федоровна,2020-12-29,2019-04-20,нет,,+7(921)902-69-52,,-,460.0,1,БДВУ,79219026952,ТС - Все Звонки,2020-12-27,79219026952,Старый
290,990534962,Коврижных Алевтина Афанасьевна,2020-11-16,2020-11-13,нет,+7(911)102-18-11,-,-,-,450.0,1,БДВУ,79111021811,ТС - Все Заявки,2020-11-09,79111021811,Новый


In [41]:
au_telefons_final = au_telefons_final[au_telefons_final['Дата'] <= au_telefons_final['Дата приема']]

In [42]:
au_telefons_final['Уникальный клиент'] = (~au_telefons_final.duplicated(subset=['Номер карты'], keep='first')).astype(int)


C:\Users\AmanovRA\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [43]:
au_telefons_final.to_excel('Обработано.xlsx', index=False)